In [87]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import folium

In [88]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [89]:
soup = BeautifulSoup(result.content, 'html5lib')

In [90]:
table = soup.find_all('table')[0] 
table
dfs = pd.read_html(str(table))
df = pd.concat(dfs)
df.columns = ['Postalcode', 'Borough','Neighbourhood']
df = df.iloc[1:]
df=df[df['Neighbourhood']!='Not assigned']
df.head()
dfgrp=df.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(''.join).reset_index()
dfgrp.head(11)

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,East Birchmount ParkIonviewKennedy Park
7,M1L,Scarborough,ClairleaGolden MileOakridge
8,M1M,Scarborough,CliffcrestCliffsideScarborough Village West
9,M1N,Scarborough,Birch CliffCliffside West


In [91]:
dfspatial=pd.read_csv("C:\Geospatial_Coordinates.csv")

In [92]:
dfoutput=dfgrp.merge(dfspatial,on='Postalcode',how='inner')

In [93]:
dfoutput.head(11)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,RougeMalvern,43.806686,-79.194353
1,M1C,Scarborough,Highland CreekRouge HillPort Union,43.784535,-79.160497
2,M1E,Scarborough,GuildwoodMorningsideWest Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,East Birchmount ParkIonviewKennedy Park,43.727929,-79.262029
7,M1L,Scarborough,ClairleaGolden MileOakridge,43.711112,-79.284577
8,M1M,Scarborough,CliffcrestCliffsideScarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch CliffCliffside West,43.692657,-79.264848


In [94]:
dfToronto=dfoutput[dfoutput['Borough'].str.contains("Toronto")]

In [95]:
dfToronto.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,The Danforth WestRiverdale,43.679557,-79.352188
42,M4L,East Toronto,The Beaches WestIndia Bazaar,43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [96]:
dfToronto.Longitude.dtype

dtype('float64')

In [97]:
venues_map = folium.Map(location=[43.676357, -79.293031], zoom_start=10)

In [98]:
from folium import plugins
from folium.plugins import MarkerCluster
from IPython.display import display
marker_cluster = MarkerCluster().add_to(venues_map)





In [99]:
for index, row in dfToronto.iterrows():    
    folium.Marker(
    [row['Latitude'], row['Longitude']],
    radius=10,
    color='red',
    popup="M4N",
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6).add_to(marker_cluster)
    venues_map.save('venues.html')

In [100]:
venues_map